In [1]:
import os
import sys
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_chrono_split
from recommenders.evaluation.python_evaluation import (
    map, ndcg_at_k, precision_at_k, recall_at_k
)
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.utils.notebook_utils import store_metadata

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.11.10 (main, Oct  3 2024, 02:26:51) [Clang 14.0.6 ]
Pandas version: 2.2.2
Tensorflow version: 2.18.0


NumPy, Pandas: Xử lý dữ liệu.

TensorFlow: Huấn luyện mô hình học sâu.

recommenders: Thư viện hệ thống gợi ý của Microsoft.

tf.get_logger().setLevel('ERROR'): Ẩn các cảnh báo, chỉ hiển thị lỗi.

Timer: Đo thời gian huấn luyện.

NCF: Mô hình Neural Collaborative Filtering.

NCFDataset: Xử lý dữ liệu đầu vào.

movielens: Bộ dữ liệu phim MovieLens-100k.

python_chrono_split: Chia dữ liệu theo thứ tự thời gian.

map, ndcg_at_k, precision_at_k, recall_at_k: Các thước đo đánh giá.

In [3]:
## top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 50
BATCH_SIZE = 32

SEED = DEFAULT_SEED

TOP_K: Số lượng phim đề xuất cho mỗi người dùng.

MOVIELENS_DATA_SIZE: Sử dụng tập dữ liệu MovieLens-100k.

EPOCHS: Huấn luyện trong 50 vòng.

BATCH_SIZE: Kích thước batch = 32.

In [4]:
df = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating", "timestamp"]
)

df.head()

INFO:recommenders.datasets.download_utils:Downloading https://files.grouplens.org/datasets/movielens/ml-100k.zip
100%|██████████| 4.81k/4.81k [00:04<00:00, 962KB/s]  


,userID,itemID,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [5]:
train, test = python_chrono_split(df, 0.75)

Chia dữ liệu:

75% tập train, 25% tập test.

Đảm bảo các user và item trong test phải xuất hiện trong train.

In [6]:
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

In [7]:
leave_one_out_test = test.groupby("userID").last().reset_index()

In [8]:
train_file = "test_train_100K/train_100.csv"
test_file = "test_train_100K/test_100.csv"
leave_one_out_test_file = "test_train_100K/leave_one_out_test_100.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)
leave_one_out_test.to_csv(leave_one_out_test_file, index=False)

In [31]:
# Đọc file CSV huấn luyện
df_train = pd.read_csv(train_file)

# Kiểm tra dữ liệu
print(df_train.head())

# Tách dữ liệu thành đầu vào và nhãn
user_input = df_train["userID"].values
item_input = df_train["itemID"].values
labels = df_train["rating"].values  # Nếu cần nhị phân hóa: labels = (labels > 0).astype(int)

print(user_input.shape, item_input.shape, labels.shape)

   userID  itemID  rating  timestamp
0       1     168     5.0  874965478
1       1     172     5.0  874965478
2       1     165     5.0  874965518
3       1     156     4.0  874965556
4       1     196     5.0  874965677
(74992,) (74992,) (74992,)


In [19]:
data = NCFDataset(train_file=train_file, test_file=leave_one_out_test_file, seed=SEED, overwrite_test_file_full=True)


INFO:recommenders.models.ncf.dataset:Indexing test_train_100K/train_100.csv ...
INFO:recommenders.models.ncf.dataset:Indexing test_train_100K/leave_one_out_test_100.csv ...
INFO:recommenders.models.ncf.dataset:Creating full leave-one-out test file test_train_100K/leave_one_out_test_100_full.csv ...
  0%|          | 0/943 [00:00<?, ?it/s]/Users/chi.nguyenth/miniconda3/envs/myenv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/chi.nguyenth/miniconda3/envs/myenv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/chi.nguyenth/miniconda3/envs/myenv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapax

In [20]:
model = NCF(
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

/Users/chi.nguyenth/miniconda3/envs/myenv/lib/python3.11/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [40]:
with Timer() as train_time: model.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [6.74s]: train_loss = 0.248532 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 20 [6.36s]: train_loss = 0.248444 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 30 [6.36s]: train_loss = 0.248369 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 40 [7.97s]: train_loss = 0.248583 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 50 [6.49s]: train_loss = 0.248192 


Took 339.42417216597823 seconds for training.


In [41]:
predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]


predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()

,userID,itemID,prediction
0,1.0,149.0,0.038095
1,1.0,88.0,0.779330
2,1.0,103.0,0.027045
3,1.0,239.0,0.457809
4,1.0,110.0,0.068867


In [42]:
with Timer() as test_time:

    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item) 
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time.interval))

Took 5.377629459020682 seconds for prediction.


In [43]:
eval_map = map(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.051904
NDCG:	0.204144
Precision@K:	0.182609
Recall@K:	0.103452


In [44]:
k = TOP_K

ndcgs = []
hit_ratio = []

for b in data.test_loader():
    user_input, item_input, labels = b
    output = model.predict(user_input, item_input, is_list=True)

    output = np.squeeze(output)
    rank = sum(output >= output[0])
    if rank <= k:
        ndcgs.append(1 / np.log(rank + 1))
        hit_ratio.append(1)
    else:
        ndcgs.append(0)
        hit_ratio.append(0)

eval_ndcg = np.mean(ndcgs)
eval_hr = np.mean(hit_ratio)

print("HR:\t%f" % eval_hr)
print("NDCG:\t%f" % eval_ndcg)

HR:	0.524920
NDCG:	0.401438


Pre-training

In [45]:
model = NCF(
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="GMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

/Users/chi.nguyenth/miniconda3/envs/myenv/lib/python3.11/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [46]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

model.save(dir_name=".pretrain/GMF")

INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [5.26s]: train_loss = 0.279879 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 20 [5.40s]: train_loss = 0.274142 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 30 [5.27s]: train_loss = 0.273542 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 40 [5.31s]: train_loss = 0.272565 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 50 [5.28s]: train_loss = 0.272122 


Took 271.8893171249947 seconds for training.


In [47]:
model = NCF(
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="MLP",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

In [48]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

model.save(dir_name=".pretrain/MLP")

INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [6.09s]: train_loss = 0.283663 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 20 [5.99s]: train_loss = 0.272865 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 30 [6.39s]: train_loss = 0.267319 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 40 [6.32s]: train_loss = 0.265387 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 50 [6.07s]: train_loss = 0.262388 


Took 307.9496523750131 seconds for training.


In [49]:
with Timer() as test_time:

    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item) 
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time.interval))

Took 6.172934292000718 seconds for prediction.


In [50]:
eval_map2 = map(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg2 = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision2 = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall2 = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map2,
      "NDCG:\t%f" % eval_ndcg2,
      "Precision@K:\t%f" % eval_precision2,
      "Recall@K:\t%f" % eval_recall2, sep='\n')

MAP:	0.045404
NDCG:	0.187634
Precision@K:	0.166808
Recall@K:	0.092421


In [ ]:
#Nếu bạn muốn train lại từ đầu,Delete pre-trained directory
#save_dir = ".pretrain"
#if os.path.exists(save_dir):
 #   shutil.rmtree(save_dir)
    
#print("Did \'%s\' exist?: %s" % (save_dir, os.path.exists(save_dir)))

Did '.pretrain' exist?: False
